In [2]:
import pandas as pd
import numpy as np
import warnings

In [3]:
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Ce dont on a besoin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV

In [5]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [6]:
chemin_bd = r"./bd_ignore/"

In [7]:
df_merge_recom = pd.read_csv(chemin_bd+"resultat2/merge_recom.csv")

In [8]:
df_merge_recom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312795 entries, 0 to 1312794
Data columns (total 19 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   adult              1312795 non-null  bool   
 1   genres_x           1312795 non-null  object 
 2   homepage           249722 non-null   object 
 3   imdb_id            1312795 non-null  object 
 4   original_language  1312795 non-null  object 
 5   original_title     1312795 non-null  object 
 6   overview           1223340 non-null  object 
 7   poster_path        1193852 non-null  object 
 8   release_date       1312795 non-null  object 
 9   revenue            1312795 non-null  int64  
 10  spoken_languages   1312795 non-null  object 
 11  tagline            380932 non-null   object 
 12  title              1312795 non-null  object 
 13  titleType          1312795 non-null  object 
 14  originalTitle      1312795 non-null  object 
 15  startYear          1312795 non-n

In [8]:
df_merge_recom

,adult,genres_x,homepage,imdb_id,original_language,original_title,overview,poster_path,release_date,revenue,spoken_languages,tagline,title,titleType,originalTitle,startYear,genres_y,primaryName,primaryProfession
0,False,"['Crime', 'Comedy']",https://www.miramax.com/movie/four-rooms/,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,4257354,['en'],Twelve outrageous guests. Four scandalous requ...,Four Rooms,movie,Four Rooms,1995.0,Comedy,Tim Roth,"actor,producer,director"
1,False,"['Crime', 'Comedy']",https://www.miramax.com/movie/four-rooms/,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,4257354,['en'],Twelve outrageous guests. Four scandalous requ...,Four Rooms,movie,Four Rooms,1995.0,Comedy,Antonio Banderas,"actor,producer,writer"
2,False,"['Crime', 'Comedy']",https://www.miramax.com/movie/four-rooms/,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,4257354,['en'],Twelve outrageous guests. Four scandalous requ...,Four Rooms,movie,Four Rooms,1995.0,Comedy,Sammi Davis,"actress,camera_department,producer"
3,False,"['Crime', 'Comedy']",https://www.miramax.com/movie/four-rooms/,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,4257354,['en'],Twelve outrageous guests. Four scandalous requ...,Four Rooms,movie,Four Rooms,1995.0,Comedy,Amanda De Cadenet,"actress,producer,director"
4,False,"['Crime', 'Comedy']",https://www.miramax.com/movie/four-rooms/,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,1995-12-09,4257354,['en'],Twelve outrageous guests. Four scandalous requ...,Four Rooms,movie,Four Rooms,1995.0,Comedy,Valeria Golino,"actress,producer,director"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312790,False,['Science Fiction'],NaN,tt27688034,ko,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,2023-08-02,0,['ko'],NaN,The Moon,movie,Deo mun,2023.0,"Action,Adventure,Drama",Park Byeong-eun,actor
1312791,False,['Science Fiction'],NaN,tt27688034,ko,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,2023-08-02,0,['ko'],NaN,The Moon,movie,Deo mun,2023.0,"Action,Adventure,Drama",Choi Byung-mo,actor
1312792,False,['Science Fiction'],NaN,tt27688034,ko,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,2023-08-02,0,['ko'],NaN,The Moon,movie,Deo mun,2023.0,"Action,Adventure,Drama",Jonathan Ehren Groff,actor
1312793,False,['Science Fiction'],NaN,tt27688034,ko,더 문,A man on Earth discovers a man in outer space....,/1tw43TQ3fgKP6QW9uyOvclWInkd.jpg,2023-08-02,0,['ko'],NaN,The Moon,movie,Deo mun,2023.0,"Action,Adventure,Drama",Paul de Havilland,"actor,producer"


In [9]:
df_merge_recom21 = df_merge_recom.drop(columns=['adult', 'original_language', 'spoken_languages', 
                                                'tagline', 'title', 'originalTitle', 'genres_x', 'revenue', 
                                                'release_date', 'primaryProfession'])

In [10]:
df_merge_recom21.to_csv(chemin_bd+"resultat2/df_merge_recom21.csv", index=False)

In [10]:
df_merge_recom22 = pd.read_csv(chemin_bd+"resultat2/df_merge_recom21.csv")

In [13]:
df_merge_recom22.isna().sum()

homepage          1063073
imdb_id                 0
original_title          0
overview            89455
poster_path        118943
titleType               0
startYear               0
genres_y                0
primaryName             0
dtype: int64

In [11]:
df_merge_recom22['homepage'] = df_merge_recom22['homepage'].fillna('None')
df_merge_recom22['poster_path'] = df_merge_recom22['poster_path'].fillna('None')
df_merge_recom22['overview'] = df_merge_recom22['overview'].fillna('None')

In [15]:
df_merge_recom22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312795 entries, 0 to 1312794
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   homepage        1312795 non-null  object 
 1   imdb_id         1312795 non-null  object 
 2   original_title  1312795 non-null  object 
 3   overview        1312795 non-null  object 
 4   poster_path     1312795 non-null  object 
 5   titleType       1312795 non-null  object 
 6   startYear       1312795 non-null  float64
 7   genres_y        1312795 non-null  object 
 8   primaryName     1312795 non-null  object 
dtypes: float64(1), object(8)
memory usage: 90.1+ MB


In [12]:
caracteristiques = ['titleType', 'startYear', 'genres_y', 'primaryName']

In [13]:
merge_filmR = df_merge_recom22[df_merge_recom22['titleType'] == 'movie']
merge_serieR = df_merge_recom22[df_merge_recom22['titleType'] == 'tvSeries']

In [ ]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  return X_encoded, SN

In [14]:
X = merge_filmR[caracteristiques]
# ne prend que les colonnes qui sont dans caracteristiques

In [22]:


X_encoded, SN = encodage_X(X, type='normalisation')

: 

In [ ]:
# on met toutes les données dans une colonne et de maniere unique avec .set

In [ ]:
df_merge_recom22 = df_merge_recom22.groupby(
    'imdb_id', sort=False, as_index=False
).agg({
    'primaryName': set,  # Affiche uniquement des valeurs uniques
    'genres_y': set,
    'startYear': set,
    'titleType': set,
    'poster_path': set,
    'overview': set,
    'original_title': set,  # Applique automatiquement l'unicité
    'homepage':set
    
})



In [ ]:
df_merge_recom22.info()

In [ ]:
df_merge_recom22['primaryName']=df_merge_recom22['primaryName'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['genres_y']=df_merge_recom22['genres_y'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['startYear']=df_merge_recom22['startYear'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['titleType']=df_merge_recom22['titleType'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['poster_path']=df_merge_recom22['poster_path'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['overview']=df_merge_recom22['overview'].apply(lambda x : str(x).replace('{', '').replace('}',''))
df_merge_recom22['original_title']=df_merge_recom22['original_title'].apply(lambda x : str(x).replace('{','').replace('}',''))
df_merge_recom22['homepage']=df_merge_recom22['homepage'].apply(lambda x : str(x).replace('{','').replace('}',''))


In [ ]:
df_merge_recom22['primaryName']=df_merge_recom22['primaryName'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['genres_y']=df_merge_recom22['genres_y'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['startYear']=df_merge_recom22['startYear'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['titleType']=df_merge_recom22['titleType'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['poster_path']=df_merge_recom22['poster_path'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['overview']=df_merge_recom22['overview'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['original_title']=df_merge_recom22['original_title'].apply(lambda x : str(x).replace("'",'').replace("'",''))
df_merge_recom22['homepage']=df_merge_recom22['homepage'].apply(lambda x : str(x).replace("'",'').replace("'",''))

In [ ]:
df_merge_recom22

In [ ]:
# standardidation des données

In [ ]:
df_merge_recom222 = df_merge_recom22.copy()

In [ ]:
df_merge_recom222.replace('nan', np.nan)

In [ ]:
df_merge_recom222.apply(lambda col: col.str.contains('NaN', na=False)).any().any()

In [ ]:
# Trouver les lignes et colonnes contenant 'NaN'
nan_positions = df_merge_recom222.apply(lambda col: col.str.contains('NaN', na=False))

# Afficher les positions
rows_with_nan = nan_positions.any(axis=1)  # Lignes avec 'NaN'
cols_with_nan = nan_positions.any(axis=0)  # Colonnes avec 'NaN'

print("Lignes contenant 'NaN':", df_merge_recom222[rows_with_nan])
print("Colonnes contenant 'NaN':", cols_with_nan[cols_with_nan].index.tolist())

In [ ]:
#df_merge_recom222.drop(row:)

In [ ]:
X= df_merge_recom22.drop(columns=['imdb_id','primaryName'])
y=	df_merge_recom22['imdb_id']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
df_merge_recom22.info()

In [ ]:
#essai5

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_merge_recom22['imdb_normalise'] = scaler.fit_transform(df_merge_recom22[['imdb_id']])

In [ ]:
# similarite

In [ ]:
RecomX= df_merge_recom22.drop(columns=['imdb_id','primaryName'])

In [ ]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  return X_encoded, SN

In [ ]:
X_encoded, SN = encodage_X()

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Création et ajustement du modèle NearestNeighbors
nn = NearestNeighbors(n_neighbors=3, metric='euclidean')
nn.fit(X_encoded)

imdb2 = ['6061', '19474']

for name in champion_legendary :
  # Recherche des films similaires à un imdb2
  df_merge_recom22.index =   df_merge_recom22[df_merge_recom22['name'] == name].index[0]
[df_merge_recom22['name'] == name].index[0] 
recom_feature = X_encoded.iloc[df_merge_recom22.index].values.reshape(1, -1)
  # Recherche des plus proches voisins
distances, indices = nn.kneighbors(recom_feature)
print(f"\nFilms les plus similaires à un {name}:")
for distance, index in zip(distances[0][1:], indices[0][1:]):
    print(f"{df_merge_recom22['Name'].iloc[index]} (Distance: {distance:.2f})")
    recom_film = df_merge_recom22.iloc[index]
    print(recom_film)

In [ ]:
#Version test ana

In [ ]:
pd.DataFrame(merge_filmR['genres_y'].unique())

In [ ]:
df_film_test = merge_filmR.copy()

In [ ]:
# supprime les lignes contenant le genre documentaire

df_film_test = df_film_test[~df_film_test.genres_y.str.contains('Documentary').dropna()]

In [ ]:
df_film_test = df_film_test[~df_film_test.genres_y.str.contains('Biography').dropna()]

In [ ]:
df_film_test2[['genre_1', 'genre_2', 'genre_3'] ]= df_film_test[df_film_test.genres_y.split(",", expand=True)]

In [55]:
# et ça échoue toujours

X2 = df_film_test[caracteristiques]
# ne prend que les colonnes qui sont dans caracteristiques

X_encoded, SN = encodage_X(X2, type='normalisation')

: 

In [17]:
# test anissa

def encodage_X2(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas


  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)


  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  #X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  X_cat_encoded = X_cat.apply(lambda col: pd.factorize(col)[0])
  #.apply(lambda x : str(x) ///df["A"] = df["A"].apply(lambda x: x-2)
  # Recodage des colonnes en valeurs numériques à l’aide de la méthode factorize titanic['sex']= titanic['sex'].factorize()[0]
  X_encoded = pd.concat([X_num_SN, X_cat_encoded], axis=1)


  return X_encoded, SN
print(X.info())  # Affiche les types et mémoire utilisée
X_encoded, SN = encodage_X2(X, type='normalisation')
print(X_encoded.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1007762 entries, 0 to 1312794
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   titleType    1007762 non-null  object 
 1   startYear    1007762 non-null  float64
 2   genres_y     1007762 non-null  object 
 3   primaryName  1007762 non-null  object 
dtypes: float64(1), object(3)
memory usage: 38.4+ MB
None


: 

In [ ]:
X_encoded